Getting started with the API:  
Set up a google account and log into google calendar with that account.   

Open https://developers.google.com/calendar/quickstart/python and click on "Enable the Google Calender API", and create a desktop project.  
Download the credentials file and save it to your computer.   

Then you need to set up a working directory below. The google api credentials json file should be saved in there. 

Import packages

In [34]:
#For the google API 
from __future__ import print_function
import datetime
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

import math

#For the Speach assistant
import speech_recognition as sr 
from gtts import gTTS
import time 
import playsound
import pyaudio

Set up speach output function

In [ ]:
def speak(text):
    tts = gTTS(text=text, lang='en')
    filename = 'voice.mp3'
    tts.save(filename)
    playsound.playsound(filename)
    os.remove(filename)
    
speak("Hello HCI class")

Set up speach input function 

In [ ]:
def get_audio():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        audio = r.listen(source)
        said = ""

        try:
            said = r.recognize_google(audio)
            print(said)
        except Exception as e:
            print("Exception: " + str(e))
    return said

Run the below code. It will open a tab where you can log into your newly created google account. It will probably stop you with the notice that "Google hasn’t verified this app". You have to click on "advanced" and then click on "Go to Quickstart(Unsafe)". It does this because you are accessing using python. Once done, wait 2 mintues, and run the code again. This time it should not open a new tab, but just print out the next event you have in your calender. That means it worked. 



In [ ]:
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']


def main():
    """Shows basic usage of the Google Calendar API.
    Prints the start and name of the next 10 events on the user's calendar.
    """
    creds = None

    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)

        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('calendar', 'v3', credentials=creds)

    # Call the Calendar API
    now = datetime.datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time
    print('Getting the upcoming 10 events')
    events_result = service.events().list(calendarId='primary', timeMin=now,
                                        maxResults=10, singleEvents=True,
                                        orderBy='startTime').execute()
    events = events_result.get('items', [])

    if not events:
        print('No upcoming events found.')
    for event in events:
        start = event['start'].get('dateTime', event['start'].get('date'))
        print(start, event['summary'])


if __name__ == '__main__':
    main()

Here is the full code all in one section. (If this gives you an error, go to your directory and delete the token files (leave credentials and voice file) and try again. 

How it works: Once you click run the bot will be listening. It is waiting for you to state: "I need assistance". When you speak those words the bot will answer "I am ready" and you can ask it:  

what plans you have on a certain date (Not in the past)
to make a note and it will ask what the note should say, you will answer and the bot will save a txt file in the WD
what the time and date is. It will give you the time and date. 

When the bot has answered, you need to wake it again with the "I need assistance" command, to make it listen again so that you can ask a new question. 

Tip: if you write your speach commands with capital letters, the bot will not understand the commands for some reason. 


In [44]:
from __future__ import print_function
import datetime
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os
import time
import pyttsx3
import speech_recognition as sr
import pytz
import subprocess
import pandas as pd #blablablabla

SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']
MONTHS = ["january", "february", "march", "april", "may", "june","july", "august", "september","october","november", "december"]
DAYS = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
DAY_EXTENTIONS = ["rd", "th", "st", "nd"]

def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

def get_audio():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        audio = r.listen(source)
        said = ""

        try:
            said = r.recognize_google(audio)
            print(said)
        except Exception as e:
            print("Exception: " + str(e))

    return said.lower()


def authenticate_google():
    """Shows basic usage of the Google Calendar API.
    Prints the start and name of the next 10 events on the user's calendar.
    """
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)

        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('calendar', 'v3', credentials=creds)

    return service


def get_events(day, service):
    
    # Call the Calendar API
    date = datetime.datetime.combine(day, datetime.datetime.min.time())
    end_date = datetime.datetime.combine(day, datetime.datetime.max.time())
    utc = pytz.UTC
    date = date.astimezone(utc)
    end_date = end_date.astimezone(utc)

    events_result = service.events().list(calendarId='primary', timeMin=date.isoformat(), timeMax=end_date.isoformat(),
                                        singleEvents=True,
                                        orderBy='startTime').execute()
    events = events_result.get('items', [])

    if not events:
        speak('No upcoming events found.')
    else:
        speak(f"You have {len(events)} events on this day.")

        for event in events:
            start = event['start'].get('dateTime', event['start'].get('date'))
            placeholder = pd.to_datetime(start).tz_localize(None)
            delta = placeholder - pd.to_datetime(datetime.datetime.now())
            hour = placeholder.hour
            minute = placeholder.minute
            print(start, event['summary'])
            start_time = str(start.split("T")[1].split("-")[0])
            if int(start_time.split(":")[0]) < 12:
                start_time = start_time + "am"
            else:
                start_time = str(int(start_time.split(":")[0])-12) + start_time.split(":")[1]
                start_time = start_time + "pm"

            speak(event["summary"] + " at " + str(hour) + " " +  str(minute))
            
            minutes = round(delta.seconds/60, 0)
            hours = int(math.floor(minutes/60))
            leftover = round(hours * 60 - minutes, 0)

            if hours == 0:
                print("that is in", minutes, "minutes")
                speak(f"that is in {minutes} minutes")
            else: 
                print("that is in", hours, "hours and", leftover, "minutes")
                speak(f"that is in,{hours} hours and, {leftover}, minutes")



            
def get_date(text):
    text = text.lower()
    today = datetime.date.today()

    if text.count("today") > 0:
        return today

    day = -1
    day_of_week = -1
    month = -1
    year = today.year

    for word in text.split():
        if word in MONTHS:
            month = MONTHS.index(word) + 1
        elif word in DAYS:
            day_of_week = DAYS.index(word)
        elif word.isdigit():
            day = int(word)
        else:
            for ext in DAY_EXTENTIONS:
                found = word.find(ext)
                if found > 0:
                    try:
                        day = int(word[:found])
                    except:
                        pass

    # THE NEW PART STARTS HERE
    if month < today.month and month != -1:  # if the month mentioned is before the current month set the year to the next
        year = year+1

    # This is slighlty different from the video but the correct version
    if month == -1 and day != -1:  # if we didn't find a month, but we have a day
        if day < today.day:
            month = today.month + 1
        else:
            month = today.month

    # if we only found a dta of the week
    if month == -1 and day == -1 and day_of_week != -1:
        current_day_of_week = today.weekday()
        dif = day_of_week - current_day_of_week

        if dif < 0:
            dif += 7
            if text.count("next") >= 0:
                dif += 7

        return today + datetime.timedelta(dif)

    if day != -1:  # FIXED FROM VIDEO
        return datetime.date(month=month, day=day, year=year)

def note(text):
    date = datetime.datetime.now()
    file_name = str(date).replace(":", "-") + "-note.txt"
    with open(file_name, "w") as f:
        f.write(text)

    subprocess.Popen(["notepad.exe", file_name])


WAKE = "I need assistance" #This is your wake command
SERVICE = authenticate_google()
print("Start")

text = "start here"
exit_words = ["stop", "cancel", "abort", "f***", "brexit", "dexit"]
check_today = ["what is today", "get today", "when is brexit"]

while all([False if t in exit_words else True for t in text.split()]):
    print("Listening")
    text = get_audio()

    if text.count(WAKE) >-1:
        speak("I am ready")
        text = get_audio()

        for phrase in check_today:
            if phrase in text:
                today = datetime.datetime.now()
                words = " ".join([str(today.strftime("%Y")), str(today.strftime("%B")), str(today.strftime("%d"))])
                speak(words)
                print(today.date)

        CALENDAR_STRS = ["what do i have", "do i have plans", "do i have plants" , "am i busy", "what are my plans for", "what are my plants for"]
        for phrase in CALENDAR_STRS:
            if phrase in text:
                date = get_date(text)
                if date:
                    get_events(date, SERVICE)
                else:
                    speak("I don't understand")

        NOTE_STRS = ["make a note", "write this down", "remember this"]
        for phrase in NOTE_STRS:
            if phrase in text:
                speak("What would you like me to write down?")
                note_text = get_audio()
                note(note_text)
                speak("I've made a note of that.")
                


        

Start
Listening
I need assistance
do I have plans today
2021-03-26T08:00:00+01:00 Programming in Julia? The beginning of the end - an online course
that is in 20 hours and -56.0 minutes
2021-03-26T12:00:00+01:00 Romcom with Victor 
that is in 56.0 minutes
Listening
Furby
Exception: 
Listening
brexit
Exception: 
Listening
cancel
recipe
Listening
cancel
cancel
